In [9]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
df=pd.read_csv("../SecondProject2/Resources/Project2_idmc_disaster_all_dataset.csv")
df.head()

,Country code,Country Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements
0,AB9,Abyei Area,2018,7/1/2018,Abyie: Flood - 01/07/2018,Weather related,Flood,2.0
1,AB9,Abyei Area,2019,6/1/2019,Abyei: Flood - southern parts - 01/06/2019,Weather related,Flood,40000.0
2,AFG,Afghanistan,2008,1/1/2008,NaN,Weather related,Extreme temperature,NaN
3,AFG,Afghanistan,2008,4/17/2008,NaN,Geophysical,Earthquake,3250.0
4,AFG,Afghanistan,2008,8/1/2008,NaN,Weather related,Flood,180.0


In [3]:
import pycountry_convert as pc

In [4]:
country_code = pc.country_name_to_country_alpha2("China", cn_name_format="default")
print(country_code)
continent_name = pc.country_alpha2_to_continent_code(country_code)
print(continent_name)

CN
AS


In [5]:
def country_code(name):
    try:
        code = pc.country_name_to_country_alpha2(name, cn_name_format="default")
        return code
    except:
        return "N/A"
def continent_name(name):
    try:
        continent = pc.country_alpha2_to_continent_code(name)
        return continent
    except:
        return "N/A"
df["Country_2D"] = df["Country Name"].apply(country_code)
df["Continent"] = df["Country_2D"].apply(continent_name)

In [6]:
df.head()

,Country code,Country Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements,Country_2D,Continent
0,AB9,Abyei Area,2018,7/1/2018,Abyie: Flood - 01/07/2018,Weather related,Flood,2.0,N/A,N/A
1,AB9,Abyei Area,2019,6/1/2019,Abyei: Flood - southern parts - 01/06/2019,Weather related,Flood,40000.0,N/A,N/A
2,AFG,Afghanistan,2008,1/1/2008,NaN,Weather related,Extreme temperature,NaN,AF,AS
3,AFG,Afghanistan,2008,4/17/2008,NaN,Geophysical,Earthquake,3250.0,AF,AS
4,AFG,Afghanistan,2008,8/1/2008,NaN,Weather related,Flood,180.0,AF,AS


In [7]:
df_Africa= df[df["Continent"]=="AF"]
df_Africa.head()

,Country code,Country Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements,Country_2D,Continent
212,AGO,Angola,2009,1/1/2009,NaN,Weather related,Flood,415.0,AO,AF
213,AGO,Angola,2009,1/1/2009,NaN,Weather related,Flood,4650.0,AO,AF
214,AGO,Angola,2009,1/1/2009,NaN,Weather related,Flood,81000.0,AO,AF
215,AGO,Angola,2010,1/1/2010,NaN,Weather related,Flood,3875.0,AO,AF
216,AGO,Angola,2010,1/1/2010,NaN,Weather related,Flood,75000.0,AO,AF


In [37]:
df_Africa.columns


Index(['Country code', 'Country Name', 'Year', 'Start Date', 'Event Name',
       'Hazard Category', 'Hazard Type', 'New Displacements', 'Country_2D',
       'Continent'],
      dtype='object')

In [8]:
df.groupby("Continent").count()

,Country code,Country Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements,Country_2D
Continent,,,,,,,,,
AF,1059,1059,1059,1059,828,1059,1059,990,1059
AS,3814,3814,3814,3814,3385,3813,3813,3632,3814
EU,517,517,517,517,394,517,517,425,517
N/A,453,453,453,453,393,453,453,419,453
NA,1251,1251,1251,1251,1067,1250,1250,1134,1251
OC,292,292,292,292,230,292,292,246,292
SA,973,973,973,973,860,972,972,924,973


In [10]:
#Import Dependencies
import os
import requests
import json
import pprint
import numpy as np
import flask
import wbdata
import datetime
import fuzzywuzzy

In [11]:
countries = ['algeria', 'angola', 'benin', 'botswana', 'burkina faso', 'burundi', 'cabo verde',
            'cameroon', 'central african republic', 'chad', 'comoros', 'congo', "cote d'ivoire",
            'djibouti', 'egypt', 'equatorial guinea', 'esqtini', 'ethiopia', 'gabon', 'gambia',
            'ghana', 'guinea', 'guinea-bissau', 'kenya', 'lesotho', 'liberia', 'libya', 'madagascar',
            'malawi', 'mali', 'mauritania', 'mauritius', 'morocco', 'mozambique', 'namibia', 'niger',
            'nigeria', 'rwanda', 'sao tome and principe', 'senegal', 'seychelles', 'sirre leone',
            'somalia', 'south africa', 'south sudan', 'sudan', 'tanzania', 'togo', 'tunisia', 
            'uganda', 'zambia', 'zimbabwe']
country_codes = ['AGO', 'ALB', 'ARB', 'BDI', 'BEN', 'BFA', 'BMN', 'BSS', 'BWA', 'CAA', 'CAF',
                'CIV', 'CME', 'CMR', 'COG', 'COM', 'CPV', 'DJI', 'DMN', 'DSF', 'DSS', 'DZA',
                'EGY', 'ETH', 'GAB', 'GHA', 'GMB', 'GNB', 'GNQ', 'KEN', 'LBR', 'LSO', 'MAR',
                'MDG', 'MEA', 'MLI', 'MNA', 'MOZ', 'MRT', 'MUS', 'MWI', 'NAF', 'NAM', 'NER',
                'NGA', 'NLS', 'NRS', 'RRS', 'RSO', 'RWA', 'SDN', 'SLE', 'SOM', 'SSA', 'SSD',
                'SSF', 'SWZ', 'SXZ', 'SYC', 'TCD', 'TGO', 'TMN', 'TSS', 'TUN', 'TZA', 'UGA',
                'XZN', 'ZAF', 'ZMB', 'ZWE']
indicators = "SP.POP.TOTL"
data_date = datetime.datetime(2008, 1, 1), datetime.datetime(2019, 1, 1)
wbdata.get_indicator(source=50)

id              name
--------------  -----------------------
SP.POP.TOTL     Population, total
SP.POP.TOTL.ZS  Population (% of total)

In [12]:
wbdata.search_countries('')

id    name
----  --------------------------------------------------------------------------------
ABW   Aruba
AFG   Afghanistan
AFR   Africa
AGO   Angola
ALB   Albania
AND   Andorra
ANR   Andean Region
ARB   Arab World
ARE   United Arab Emirates
ARG   Argentina
ARM   Armenia
ASM   American Samoa
ATG   Antigua and Barbuda
AUS   Australia
AUT   Austria
AZE   Azerbaijan
BDI   Burundi
BEA   East Asia & Pacific (IBRD-only countries)
BEC   Europe & Central Asia (IBRD-only countries)
BEL   Belgium
BEN   Benin
BFA   Burkina Faso
BGD   Bangladesh
BGR   Bulgaria
BHI   IBRD countries classified as high income
BHR   Bahrain
BHS   Bahamas, The
BIH   Bosnia and Herzegovina
BLA   Latin America & the Caribbean (IBRD-only countries)
BLR   Belarus
BLZ   Belize
BMN   Middle East & North Africa (IBRD-only countries)
BMU   Bermuda
BOL   Bolivia
BRA   Brazil
BRB   Barbados
BRN   Brunei Darussalam
BSS   Sub-Saharan Africa (IBRD-only countries)
BTN   Bhutan
BWA   Botswana
CAA   Sub-Saharan Africa (IFC classif

In [33]:
data = wbdata.get_data(indicators, country = country_codes, data_date = data_date)
df_wbdata = pd.DataFrame(data)
df_wbdata = df_wbdata.rename(columns={"indicator": "Indicator", 
                  "country": "Country",
                  "countryiso3code": "Country code",
                  "date": "Year",
                  "value": "Population",})
df_wbdata = df_wbdata.filter(items = ['Country', 'Country code', 'Year', 'Population'])
df_wbdata.dropna(inplace=True)
df_wbdata['Country'] = df_wbdata['Country'].astype(str)
df_wbdata['Country code'] = df_wbdata['Country code'].astype(str)
df_wbdata['Year'] = df_wbdata['Year'].astype(str)
df_wbdata['Population'] = df_wbdata['Population'].astype(str)
df_wbdata['Country'] = df_wbdata['Country'].str.slice(23, -2)
df_wbdata.head()

,Country,Country code,Year,Population
0,Angola,AGO,2019,31825295
1,Angola,AGO,2018,30809762
2,Angola,AGO,2017,29816748
3,Angola,AGO,2016,28842484
4,Angola,AGO,2015,27884381


In [34]:
df_wbdata.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 672 entries, 0 to 671
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country       672 non-null    object
 1   Country code  672 non-null    object
 2   Year          672 non-null    object
 3   Population    672 non-null    object
dtypes: object(4)
memory usage: 26.2+ KB


In [36]:
df_wbdata.columns


Index(['Country', 'Country code', 'Year', 'Population'], dtype='object')

In [50]:
df_Africa['Year'] = df_Africa['Year'].astype('int64')
df_wbdata['Year'] = df_wbdata['Year'].astype('int64')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
merged_df = pd.merge (left = df_Africa, right = df_wbdata, how = "left", on=['Country code','Year'])
merged_df

,Country code,Country Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements,Country_2D,Continent,Country,Population
0,AGO,Angola,2009,1/1/2009,NaN,Weather related,Flood,415.0,AO,AF,Angola,22514281
1,AGO,Angola,2009,1/1/2009,NaN,Weather related,Flood,4650.0,AO,AF,Angola,22514281
2,AGO,Angola,2009,1/1/2009,NaN,Weather related,Flood,81000.0,AO,AF,Angola,22514281
3,AGO,Angola,2010,1/1/2010,NaN,Weather related,Flood,3875.0,AO,AF,Angola,23356246
4,AGO,Angola,2010,1/1/2010,NaN,Weather related,Flood,75000.0,AO,AF,Angola,23356246
...,...,...,...,...,...,...,...,...,...,...,...,...
1054,ZWE,Zimbabwe,2018,1/1/2018,Zimbabwe: Hailstorm - Umguza - 01/01/2018,Weather related,Storm,80.0,ZW,AF,Zimbabwe,14439018
1055,ZWE,Zimbabwe,2019,3/4/2019,"Malawi, Mozambique, Madagascar, Zimbabwe: Cycl...",Weather related,Storm,50905.0,ZW,AF,Zimbabwe,14645468
1056,ZWE,Zimbabwe,2019,2/13/2019,Zimbabwe: Floods- Chiredzi (Masvingo)- 13/02/2019,Weather related,Flood,200.0,ZW,AF,Zimbabwe,14645468
1057,ZWE,Zimbabwe,2019,10/7/2019,Zimbabwe: Hailstorm- Masvingo (Zaka)- 07/10/2019,Weather related,Storm,400.0,ZW,AF,Zimbabwe,14645468


In [56]:
merged_df[merged_df['Population'].isna()].count()

Country code         33
Country Name         33
Year                 33
Start Date           33
Event Name           22
Hazard Category      33
Hazard Type          33
New Displacements    29
Country_2D           33
Continent            33
Country               0
Population            0
dtype: int64